In [1]:
%run setup.py

In [2]:
import asyncio

In [3]:
from dotenv import load_dotenv

In [4]:
from chatflare.prompt.base import PromptTemplate
from chatflare.core.llm_wrapper import BaseChain
from chatflare.model.openai import ChatOpenAI
from chatflare.model.llama_bedrock import LlamaBedrock

In [5]:
model = LlamaBedrock()

In [6]:
load_dotenv("../bedrock.env")

True

In [7]:
!pwd

/Users/qiurui/Desktop/research/2023 further/perspecto-cards/server/ChatFlare


In [8]:
!ls

LICENSE              chatflare.egg-info   setup.py
README.md            docs                 test_history_control
assets               notebooks            tests
chatflare            requirements-dev.txt
chatflare-venv       requirements.txt


In [9]:
import tiktoken
from PyPDF2 import PdfReader
pdf_path = "./tests/example_docs/sample_med2.pdf"
with open(pdf_path, 'rb') as file:
    reader = PdfReader(file)
    num_pages = len(reader.pages)
    text = ""
    inside_section = False
    pages = []
    for i in range(num_pages):
        page = reader.pages[i]
        page_text = page.extract_text()
        pages.append(page_text)

sumLength = 0
for page in pages: 
    sumLength += len(page)

summed_doc = "\n".join(pages)

In [10]:
READ_PROMPT_PREFIX = """You are an research agent designed to help human perform an literature review tasks from a set of documents.
You are give a task to review information of the given research question: {{query}}.
You are given a paper to read, and you need to summarize the information relevant to the given topic and also the overall information of the document that you feel important or interesting or worth to dive in. 
{{paper_already_read}}
{{findings_so_far}}
The paper you need to read is below and is wrapped between a pair of triple backticks:  
```
{{paper_to_read}}
```
Be careful that the paper you are assigned might not relevant to the research question: {{query}}. But if it is relevant, you also are required to generate a overall thought to reflect your understanding of the given topic `{{query}}`so far, based on this paper, papers you have already read and your findings. 
Be careful about the conflict findings from this paper and the papers you have read (if you have), do not judge which one is more reasonable based on our understanding, just write it to your thought and wait for further exploration!
{{inspiration_conversation_history}}

{RESPONSE_FORMAT}
"""

READ_RESPONSE_FORMAT = """
When responding use a markdown code snippet with a JSON object formatted in the \
following schema:
```json
{{  
    "findings_of_the_paper": str, //the information in the paper that directly related to the given research question `{{query}}`, return `not included` if it does not contain the information that can directly relate to the research question
    "related_to_query": bool, //whether the paper is directly related to the given research question `{{query}}`, 
    "reason_of_exclusion": str, //if your answer was `false` in "related_to_query", you need to provide the reason why you think it is not related to the research question `{{query}}
    "summary_of_the_paper": str, //summary of the paper given to you, include the information that you feel important and worth further exploration
    "thought": str, //your overall understanding of question `{{query}}` so far, not just from this paper but also your previous findings. Do not include irrelevant information here. Pay attention to the length of your thought, it should be no longer than 500 words.    
}}
```
Return only the makrdown and Nothing Else!
"""

In [11]:
READ_PROMPT_PREFIX = """You are an research agent designed to help human perform an literature review tasks from a set of documents.
You are give a task to review information of the given research question: {{query}}.
You are given a paper to read, and you need to summarize the information relevant to the given topic and also the overall information of the document that you feel important or interesting or worth to dive in. 
{{paper_already_read}}
{{findings_so_far}}
The paper you need to read is below and is wrapped between a pair of triple backticks:  
```
{{paper_to_read}}
```
Be careful that the paper you are assigned might not relevant to the research question: {{query}}. But if it is relevant, you also are required to generate a overall thought to reflect your understanding of the given topic `{{query}}`so far, based on this paper, papers you have already read and your findings. 
Be careful about the conflict findings from this paper and the papers you have read (if you have), do not judge which one is more reasonable based on our understanding, just write it to your thought and wait for further exploration!
{{inspiration_conversation_history}}

{RESPONSE_FORMAT}
"""

READ_RESPONSE_FORMAT = """
When responding use a markdown code snippet with a JSON object formatted in the \
following schema:
```json
{{  
    "findings_of_the_paper": str, //the information in the paper that directly related to the given research question `{{query}}`, return `not included` if it does not contain the information that can directly relate to the research question
}}
```
Return only the makrdown and Nothing Else!
"""

In [12]:
prompt_prefix = READ_PROMPT_PREFIX.format(RESPONSE_FORMAT=READ_RESPONSE_FORMAT)

prompt = PromptTemplate(template=prompt_prefix)

In [13]:
prompt.variables

['query',
 'paper_already_read',
 'findings_so_far',
 'paper_to_read',
 'query',
 'query',
 'inspiration_conversation_history']

In [14]:
chain = BaseChain(model, prompt)

In [16]:
query = "what's the summary of this paper"
paper_already_read = ""
findings_so_far = ""
paper_to_read = summed_doc[:200]
inspiration_conversation_history = ""

In [17]:
chain.turn_on_json_mode()

In [18]:
res = chain.predict(query=query, paper_already_read=paper_already_read, findings_so_far=findings_so_far, paper_to_read=paper_to_read, inspiration_conversation_history=inspiration_conversation_history)

In [19]:
import json

In [20]:
json.loads(res)

{'findings_of_the_paper': 'not included'}

In [17]:
res_2 = await chain.apredict(query=query, paper_already_read=paper_already_read, findings_so_far=findings_so_far, paper_to_read=paper_to_read, inspiration_conversation_history=inspiration_conversation_history)

In [21]:
json.loads(res_2)

{'findings_of_the_paper': 'The paper investigates the administration time-dependent antihypertensive efficacy of telmisartan in patients with essential hypertension. It compares the efficacy of morning versus evening administration of telmisartan, with 215 patients randomly assigned to receive telmisartan (80 mg/d) as a monotherapy either on awakening or at bedtime. The study aims to determine if the administration time-dependent efficacy is a class-related feature of all angiotensin receptor blockers or specific to valsartan, given that telmisartan has a distinct terminal half-life compared to other angiotensin receptor blockers.'}

In [22]:
res_2

'{  \n    "findings_of_the_paper": "The paper investigates the administration time-dependent antihypertensive efficacy of telmisartan in patients with essential hypertension. It compares the efficacy of morning versus evening administration of telmisartan, with 215 patients randomly assigned to receive telmisartan (80 mg/d) as a monotherapy either on awakening or at bedtime. The study aims to determine if the administration time-dependent efficacy is a class-related feature of all angiotensin receptor blockers or specific to valsartan, given that telmisartan has a distinct terminal half-life compared to other angiotensin receptor blockers."\n}\n         '